In [1]:
import numpy as np
import pandas as pd

In [4]:
# pull prior split
oldsplit = pd.read_csv('d:/Dropbox (Partners Healthcare)/profile_9-2022/derived_data/split_9-2022.csv')

oldsplit.head()
oldsplit.dfci_mrn.nunique()

393588

In [3]:
new_dataset = pd.read_csv('d:/Dropbox (Partners Healthcare)/profile_3-2023/structured_data/REQ_KK71_105304_2_PT_INFO_STATUS_REGISTRATION.csv', low_memory=False)[['DFCI_MRN']].rename(columns={'DFCI_MRN':'dfci_mrn'})
new_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56870 entries, 0 to 56869
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   dfci_mrn  56870 non-null  int64
dtypes: int64(1)
memory usage: 444.4 KB


In [7]:
new_dataset = new_dataset.groupby('dfci_mrn').first().reset_index()
new_dataset.shape[0]

56869

In [8]:
new_patients = new_dataset[~new_dataset.dfci_mrn.isin(oldsplit.dfci_mrn)]

In [9]:
new_patients.shape[0]

2133

In [11]:

new_true_test_mrns = new_patients.sample(frac=0.1, random_state=1)
remainder_mrns = new_patients[new_patients['dfci_mrn'].isin(new_true_test_mrns.dfci_mrn) == False]
new_validation_mrns = remainder_mrns.sample(frac=0.111111, random_state=2)
new_training_mrns = remainder_mrns[remainder_mrns['dfci_mrn'].isin(new_validation_mrns.dfci_mrn) == False]

In [12]:
new_true_test_mrns['split'] = 'test'
new_validation_mrns['split'] = 'validation'
new_training_mrns['split'] = 'train'

<ipython-input-12-db97fcda6dff>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_training_mrns['split'] = 'train'


In [13]:
new_split = pd.concat([new_training_mrns, new_validation_mrns, new_true_test_mrns], axis=0)

In [14]:
new_split.dfci_mrn.isin(oldsplit.dfci_mrn).value_counts()

False    2133
Name: dfci_mrn, dtype: int64

In [15]:
output = pd.concat([oldsplit[['dfci_mrn','split']], new_split], axis=0).sort_values(by='dfci_mrn').reset_index(drop=True)

In [16]:
output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395721 entries, 0 to 395720
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   dfci_mrn  395721 non-null  float64
 1   split     395721 non-null  object 
dtypes: float64(1), object(1)
memory usage: 6.0+ MB


In [17]:
output.to_csv('d:/Dropbox (Partners Healthcare)/profile_3-2023/derived_data/split_3-2023.csv')

In [18]:
oldsplit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393588 entries, 0 to 393587
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  393588 non-null  int64  
 1   dfci_mrn    393588 non-null  float64
 2   split       393588 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 9.0+ MB


In [19]:
output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395721 entries, 0 to 395720
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   dfci_mrn  395721 non-null  float64
 1   split     395721 non-null  object 
dtypes: float64(1), object(1)
memory usage: 6.0+ MB


In [21]:
output.split.value_counts()

train         316577
test           39572
validation     39572
Name: split, dtype: int64